# Imports

In [1]:
# Append location to path to allow custom modules to be used.
import sys, os
sys.path.append(os.path.abspath(os.path.join("..", "..")))

In [2]:
import cgael
from cgael.models.SimpleColor import *
from cgael.models.extras.LanguageDiscriminator import *
from cgael.metrics import brevity

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layer

import pygad
import pygad.kerasga

import numpy as np
import pandas as pd

from PIL import Image

# Set Up

## Folders

In [3]:
root_folder = os.path.join("C:",os.sep,"Users","nicho","PyProjects","CGAEL_Results")
experiment_folder = "single_simple_color_style"
experiment_prefix = "sscs"

In [4]:
os.makedirs(os.path.join(root_folder, experiment_folder), exist_ok=True)

In [5]:
ID_FORMAT = "{experiment_prefix}_{i}"

i = 1
while os.path.exists(os.path.join(root_folder, experiment_folder, f"{ID_FORMAT.format(experiment_prefix=experiment_prefix, i=i)}.npy")):
    i += 1
    
experiment_id = ID_FORMAT.format(experiment_prefix=experiment_prefix, i=i)
filename = f"{experiment_id}.npy"
filepath = os.path.join(root_folder, experiment_folder, filename)
print(filepath)

C:\Users\nicho\PyProjects\CGAEL_Results\single_simple_color_style\sscs_1.npy


## Discriminator

In [6]:
weights_file = os.path.join(root_folder, "discriminator", "eng_L10_1.npy")
discrim = LanguageDiscriminatorModel(word_length=10, compile=False)
discrim.model.set_weights(np.load(weights_file, allow_pickle=True))

In [7]:
def style_loss(data):
    """
    As the trained discriminator gauges values on the range of [0, 1]
    where 1 is real and 0 is fake, the values need to be inverted to
    serve as a loss function.
    """
    return 1-discrim(data)

# Training

In [8]:
ts = cgael.LanguageTokenSet("CHAT", '-')
gen = SimpleColorGenerator(
    [Swatch.WHITE, Swatch.BLACK, Swatch.RED, Swatch.GREEN, Swatch.BLUE, Swatch.YELLOW, Swatch.CYAN, Swatch.MAGENTA], 
    blur=0, batch_lock=True)
model = SimpleColorModel(
    token_set=ts, 
    word_count=1, 
    word_length=10, 
    #brevity_function=brevity.simple_brevity, 
    #brevity_weight=.25,
    style_function=style_loss,
    style_weight=.25,
)

In [22]:
msg1 = ts.encode("CATCH THAT CAT", (4, 10))
msg1

<tf.Tensor: shape=(4, 10), dtype=int64, numpy=
array([[1, 3, 4, 1, 2, 0, 0, 0, 0, 0],
       [4, 2, 3, 4, 0, 0, 0, 0, 0, 0],
       [1, 3, 4, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)>

In [23]:
msg2 = ts.encode("THAT CATH HAT", (4, 10))
msg2

<tf.Tensor: shape=(4, 10), dtype=int64, numpy=
array([[4, 2, 3, 4, 0, 0, 0, 0, 0, 0],
       [1, 3, 4, 2, 0, 0, 0, 0, 0, 0],
       [2, 3, 4, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)>

In [24]:
batch = tf.convert_to_tensor([msg1, msg2])
batch

<tf.Tensor: shape=(2, 4, 10), dtype=int64, numpy=
array([[[1, 3, 4, 1, 2, 0, 0, 0, 0, 0],
        [4, 2, 3, 4, 0, 0, 0, 0, 0, 0],
        [1, 3, 4, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[4, 2, 3, 4, 0, 0, 0, 0, 0, 0],
        [1, 3, 4, 2, 0, 0, 0, 0, 0, 0],
        [2, 3, 4, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]], dtype=int64)>

In [43]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

In [39]:
a = tf.reshape(batch, (-1, 10))
a

<tf.Tensor: shape=(8, 10), dtype=int64, numpy=
array([[1, 3, 4, 1, 2, 0, 0, 0, 0, 0],
       [4, 2, 3, 4, 0, 0, 0, 0, 0, 0],
       [1, 3, 4, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [4, 2, 3, 4, 0, 0, 0, 0, 0, 0],
       [1, 3, 4, 2, 0, 0, 0, 0, 0, 0],
       [2, 3, 4, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)>

In [40]:
b = discrim(a)
b

<tf.Tensor: shape=(8, 1), dtype=float32, numpy=
array([[9.9852735e-01],
       [9.9921155e-01],
       [9.9754417e-01],
       [7.7029401e-03],
       [9.9921155e-01],
       [4.4176895e-06],
       [9.9208403e-01],
       [7.7029401e-03]], dtype=float32)>

In [45]:
tf.print(b, summarize=-1)

[[0.998527348]
 [0.99921155]
 [0.997544169]
 [0.00770294]
 [0.99921155]
 [4.41768952e-06]
 [0.992084]
 [0.00770294]]


In [41]:
c = tf.math.reduce_mean(b)
c

<tf.Tensor: shape=(), dtype=float32, numpy=0.6252486>

In [31]:
x = batch.numpy().reshape((8, 10))
x

array([[1, 3, 4, 1, 2, 0, 0, 0, 0, 0],
       [4, 2, 3, 4, 0, 0, 0, 0, 0, 0],
       [1, 3, 4, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [4, 2, 3, 4, 0, 0, 0, 0, 0, 0],
       [1, 3, 4, 2, 0, 0, 0, 0, 0, 0],
       [2, 3, 4, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [32]:
y = style_loss(x)
y

<tf.Tensor: shape=(8, 1), dtype=float32, numpy=
array([[1.4726520e-03],
       [7.8845024e-04],
       [2.4558306e-03],
       [9.9229705e-01],
       [7.8845024e-04],
       [9.9999559e-01],
       [7.9159737e-03],
       [9.9229705e-01]], dtype=float32)>

In [ ]:
tf.

In [9]:
ga_inst = model.train(
    generator = gen,
    generations = 80,
    num_solutions = 50,
    num_parents_mating = 5,
    mutation_percent_genes = .1,
)

c:\Users\nicho\PyProjects\CGAEL\.conda\Lib\site-packages\pygad\pygad.py:909: UserWarning: The steady-state parent (sss) selection operator is used despite that no parents are kept in the next generation.
  warnings.warn("The steady-state parent (sss) selection operator is used despite that no parents are kept in the next generation.")
Input 0 of layer "model" is incompatible with the layer: expected shape=(None, 10), found shape=(8, 1, 10)
Traceback (most recent call last):
  File "c:\Users\nicho\PyProjects\CGAEL\.conda\Lib\site-packages\pygad\pygad.py", line 1688, in cal_pop_fitness
    fitness = self.fitness_func(self, sol, sol_idx)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nicho\PyProjects\CGAEL\cgael\models\SimpleColor.py", line 133, in _fitness
    loss_style = 0 if self.style_function is None else self.style_function(pred_lang).numpy() * self.style_weight
                                                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Us

AttributeError: 'tuple' object has no attribute 'tb_frame'

In [ ]:
np.save(filepath, np.array(model.model.get_weights(), dtype="object"))

# Evaluation

In [ ]:
plot = ga_inst.plot_fitness(title=experiment_id, color="#0C69D3")
plot.savefig(os.path.join(root_folder, experiment_folder, f"{experiment_id}.png"))

In [ ]:
KEY_SWATCH = "swatch"
KEY_TEXT = "text"
KEY_IN = "input"
KEY_OUT = "output"
KEY_ROUND = "output_rounded"

swatches = [Swatch.BLACK, Swatch.RED, Swatch.GREEN, Swatch.YELLOW, Swatch.BLUE, Swatch.MAGENTA, Swatch.CYAN, Swatch.WHITE]
samples = np.array([[sample_swatch(x)] for x in swatches])
lang, out = model.model(samples)
text = [ts.decode(x) for x in lang]

d = {x:[] for x in [KEY_SWATCH, KEY_TEXT, KEY_IN, KEY_OUT, KEY_ROUND]}
for s, t, i, o in zip(swatches, text, samples, out):
    d[KEY_SWATCH].append(s)
    d[KEY_TEXT].append(t)
    d[KEY_IN].append(i)
    o = o.numpy()
    d[KEY_OUT].append(o)
    o = np.round(o)
    d[KEY_ROUND].append(o)
df = pd.DataFrame(data=d)
display(df)

In [ ]:
KEY_TEXT = "text"
KEY_OUT = "output"
KEY_ROUND = "output_rounded"

def generate_words(tokens, n):
    temp = tokens
    total = [''] + temp
    for _ in range(n-1):
        temp = [x+y for x in temp for y in tokens]
        total = total + temp
    return total

text = generate_words(ts.alphabet_tokens, 3)
data = np.array([ts.encode(x, shape=(1,3)) for x in text])
out = model.listener(data)

d = {x:[] for x in [KEY_TEXT, KEY_OUT, KEY_ROUND]}
for t, o in zip(text, out):
    d[KEY_TEXT].append(t)
    o = o.numpy()
    d[KEY_OUT].append(o)
    o = np.round(o)
    d[KEY_ROUND].append(o)
df = pd.DataFrame(data=d)
display(df)